# Genie Top 200 Chart

In [18]:
import requests
import pandas as pd

In [19]:
# Genie Top 200
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url) #json 안붙는거는 json으로 가지고 오지 않기 때문
html = req.text
html #블럭되었다

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [21]:
#chrome user agent 검색 : header user-agent
header = {'User-Agent' :    
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}
req = requests.get(url, headers=header)
html = req.text

In [24]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html,'html.parser')

In [27]:
table = soup.select_one('table.list-wrap')
trs = table.select('tr.list')
len(trs)

50

In [30]:
trs = soup.select('tr.list')
len(trs)

50

## 3. 여러개의 데이중 하나를 선택해서 원하는 정보 추출
- rank, title, album 찾기

In [31]:
tr = trs[0]
#rank
tr.select_one('.number').get_text() #지저분하게 나온다

'1\n                                        \n                                    \n유지\n\n'

In [32]:
tr.select_one('.number').get_text().split('\n')[0]

'1'

In [34]:
rank = int(tr.select_one('.number').get_text().split('\n')[0])
rank

1

In [35]:
# title 찾기
tr.select_one('.info').find('a').get_text()

'\n                                        \n                                            \n                                        \n                                        \n                                        \n                                        \n                                            \n                                                Queendom'

In [36]:
title = tr.select_one('.info').find('a').get_text().strip()
title

'Queendom'

In [38]:
# 다른방식으로 타이틀가져오기
title = tr.select_one('.info').select_one('.title').get_text().strip()
title

'Queendom'

In [39]:
artist = tr.select_one('.info').select_one('.artist').get_text().strip()
artist

'Red Velvet (레드벨벳)'

In [42]:
# album
album = tr.select_one('.info').select_one('.albumtitle').get_text().strip()
album

'Queendom - The 6th Mini Album'

## 한페이지에 있는 모든 데이터 반복문으록 가져오기

In [43]:
rank_list, title_list, artist_list, album_list =[], [], [], []

for tr in trs:
    rank = int(tr.select_one('.number').get_text().split('\n')[0])
    title = tr.select_one('.info').find('a').get_text().strip()
    artist = tr.select_one('.info').select_one('.artist').get_text().strip()
    album = tr.select_one('.info').select_one('.albumtitle').get_text().strip()
    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)
    #이제 데이터 프레임화

In [44]:
df = pd.DataFrame({
    '순위' : rank_list,
    '곡명' : title_list,
    '가수' : artist_list,
    '앨범' : album_list
})
df.tail()

,순위,곡명,가수,앨범
45,46,사이렌 Remix (Feat. UNEDUCATED KID & Paul Blanco),호미들,사이렌 Remix
46,47,어푸 (Ah puh),아이유 (IU),IU 5th Album 'LILAC'
47,48,가을 우체국 앞에서,김대명,슬기로운 의사생활 시즌2 OST Part 2 (tvN 목요드라마)
48,49,Life Goes On,방탄소년단,BE
49,50,그대라는 사치,임영웅,사랑의 콜센타 PART62


In [46]:
lines = []
for tr in trs:
    rank = int(tr.select_one('.number').get_text().split('\n')[0])
    title = tr.select_one('.info').find('a').get_text().strip()
    artist = tr.select_one('.info').select_one('.artist').get_text().strip()
    album = tr.select_one('.info').select_one('.albumtitle').get_text().strip()
    lines.append([rank, title, artist, album])

In [47]:
df2 = pd.DataFrame(lines, columns=['순위', '곡명', '가수', '앨범'])
df2.head()

,순위,곡명,가수,앨범
0,1,Queendom,Red Velvet (레드벨벳),Queendom - The 6th Mini Album
1,2,신호등,이무진,신호등
2,3,낙하 (With 아이유),AKMU (악뮤),NEXT EPISODE
3,4,바라만 본다,MSG워너비 (M.O.M),MSG워너비 1집
4,5,Weekend,태연 (TAEYEON),Weekend


## 5. 모든페이지에서 데이터 가져오기

In [48]:
rank_list, title_list, artist_list, album_list =[], [], [], []
for page in range(1,5):
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20210817&hh=14&rtm=Y&pg={page}'
    req = requests.get(url, headers=header) #header는 위에 있다
    html = req.text
    soup = BeautifulSoup(html,'html.parser') #페이지 가져왔으면 읽어야지
    trs = soup.select('tr.list') #그다음 찾고자하는 리스트 찾아야지
    for tr in trs:
        rank = int(tr.select_one('.number').get_text().split('\n')[0])
        title = tr.select_one('.info').find('a').get_text().strip()
        artist = tr.select_one('.info').select_one('.artist').get_text().strip()
        album = tr.select_one('.info').select_one('.albumtitle').get_text().strip()
        rank_list.append(rank)
        title_list.append(title)
        artist_list.append(artist)
        album_list.append(album)

In [56]:
df = pd.DataFrame({
    '순위' : rank_list,
    '곡명' : title_list,
    '가수' : artist_list,
    '앨범' : album_list
})
df.head(5)

,순위,곡명,가수,앨범
0,1,Queendom,Red Velvet (레드벨벳),Queendom - The 6th Mini Album
1,2,신호등,이무진,신호등
2,3,낙하 (With 아이유),AKMU (악뮤),NEXT EPISODE
3,4,바라만 본다,MSG워너비 (M.O.M),MSG워너비 1집
4,5,Weekend,태연 (TAEYEON),Weekend


In [57]:
df.to_csv('GenieTop200(210817.csv', index=False)